<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/maya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

version du 30/09/2021

# QUESTIONS

### Question 1 : temps écoulé

Pour chaque visite de chaque personne, le temps écoulé depuis la première visite de cette personne. Exprimer le temps écoulé par un intervalle de temps résultant d'une différence entre deux dates. L'intervalle indique le nombre de mois et le nombre de jours.

Le schema attendu est (photoID , personID, date, datePremiereVisiste, temps_ecoule)

Astuce: commencer par compléter chaque visite avec la date de la première visite de la personne.

In [ ]:
%sql
with premieres_dates as (
select personID, photoID, date as p_date, jour as p_jour, mois as p_mois, annee as p_annee, rank() over(partition by personID order by date) as p_rang
from VisitesDetail
), 
dernieres_dates as (
select personID, photoID, date as d_date, jour as d_jour, mois as d_mois, annee as d_annee, rank() over(partition by personID order by date desc) as d_rang
from VisitesDetail
)
select p.personID, p.photoID, d.d_date as derniere_date, p.p_date as premiere_date from premieres_dates p, dernieres_dates d where p_rang = 1 and d_rang=1 and p.PersonID = d.PersonID

personID,photoID,derniere_date,premiere_date
Alice,p16,2020-10-05,2019-12-25
Bob,p13,2020-09-04,2020-02-01
Bob,p13,2020-09-04,2020-02-01
Carole,p15,2020-02-01,2019-11-11
David,p9,2019-12-25,2019-12-25
Eva,p10,2019-12-26,2019-12-25


### Question 2: Requête avec fenêtre glissante de taille fixe

Pour chaque visite de chaque personne, le nombre de jours depuis la visite précédente de cette personne. 

Le schéma attendu est (photoID, personID, date, precedenteDate, nbJours)

Astuce: penser à utiliser la fonction datediff pour comparer deux dates et obtenir un nombre de jours

In [ ]:
%sql
select v.personId, v.photoId, v.date as d1, rank() over (partition by personID order by date desc) as r1
from VisitesDetail v

personId,photoId,d1,r1
Alice,p7,2020-10-05,1
Alice,p6,2020-09-05,2
Alice,p5,2020-09-04,3
Alice,p2,2020-09-01,4
Alice,p17,2020-02-02,5
Alice,p12,2020-02-01,6
Alice,p16,2019-12-25,7
Bob,p3,2020-09-04,1
Bob,p4,2020-09-04,1
Bob,p1,2020-09-03,3


In [ ]:
%sql
with t1 as (
select v.personId, v.photoId, v.date as d1, rank() over (partition by personID order by date desc) as r1
from VisitesDetail v
), 
t2 as (
select v.personId, v.photoID, v.date as d2, rank() over (partition by personID order by date desc) as r2
from VisitesDetail v
)
select t1.personID, t1.photoID ,t2.photoID as photoID_pred, t1.d1, t2.d2 from t1, t2 where t1.PersonId = t2.PersonId and t1.r1 = t2.r2-1

personID,photoID,photoID_pred,d1,d2
Alice,p7,p6,2020-10-05,2020-09-05
Alice,p6,p5,2020-09-05,2020-09-04
Alice,p5,p2,2020-09-04,2020-09-01
Alice,p2,p17,2020-09-01,2020-02-02
Alice,p17,p12,2020-02-02,2020-02-01
Alice,p12,p16,2020-02-01,2019-12-25
Bob,p1,p13,2020-09-03,2020-02-01
Carole,p14,p8,2020-02-01,2019-12-25
Carole,p8,p15,2019-12-25,2019-11-11
Eva,p11,p10,2019-12-26,2019-12-25


## Question 3 : Semaine glissante
Pour chaque jour de visite de chaque personne, le nombre de visite sur 7 jours glissants.

Schema (personID, date, nbVisite7jours)

In [ ]:
%sql
select v.personID, v.date, count(*) over (partition by v.personID order by v.date range between current row and 7 following) as nb
from VisitesDetail v
-- group by v.personID count(*) as nbVisite7jours,

personID,date,nb
Alice,2019-12-25,1
Alice,2020-02-01,2
Alice,2020-02-02,1
Alice,2020-09-01,3
Alice,2020-09-04,2
Alice,2020-09-05,1
Alice,2020-10-05,1
Bob,2020-02-01,1
Bob,2020-09-03,3
Bob,2020-09-04,2


## Question 4 : Trajets
On veut enrichir les visites avec une information de trajet.
Un trajet représente toutes les visites d'une personne telles que le nombre de jours entre deux visites consécutives soit inférieur à une semaine.

Schema : (photoID, personID, date, nbJours, numTrajet)

Astuces :
utiliser la réponse à la question 2. La fonction if dans la clause select peut indiquer si la visite est le début du trajet suivant ou non.

In [ ]:
%sql
-- à compléter (solution dans TP2_solutions)



### 4 b)
Ajouter la condition: un trajet ne peut pas durer plus de 3 jours au total

In [ ]:
%sql


## Question 5 : IMDB

Proposer des requetes OVER PARTITION BY  sur la base IMDB